In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os
import keras
import matplotlib.pyplot as plt
import sklearn
from keras.layers import Dense,GlobalAveragePooling2D, Flatten, Dropout
from keras.applications import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.metrics import confusion_matrix
import itertools
import sys

Using TensorFlow backend.


In [0]:
X_train = np.load("../content/gdrive/My Drive/X_train_PLS.npy")
Y_train = np.load("../content/gdrive/My Drive/Y_train_PLS.npy")
X_valid = np.load("../content/gdrive/My Drive/X_val_PLS.npy")
Y_valid = np.load("../content/gdrive/My Drive/Y_val_PLS.npy")

In [0]:
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(27388, 224, 224, 3)
(27388, 7)
(5262, 224, 224, 3)
(5262, 7)


In [0]:
#MODEL

base_model = MobileNetV2(input_shape = (224,224,3), weights = 'imagenet', include_top = False, pooling = max)

x = base_model.output

#We will add: flatten, dense, dense
x = Flatten()(x)

#This is the dense layer we want to experiment with.  Start at 256.
features = Dense(256, activation = 'relu')(x)

preds = Dense(7, activation = 'softmax')(features)

modelMNV2=Model(inputs = base_model.input, outputs = preds)

print(modelMNV2.summary())

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_____________________________________

In [0]:
for i,layer in enumerate(modelMNV2.layers):
  print(i,layer.name)

0 input_1
1 Conv1_pad
2 Conv1
3 bn_Conv1
4 Conv1_relu
5 expanded_conv_depthwise
6 expanded_conv_depthwise_BN
7 expanded_conv_depthwise_relu
8 expanded_conv_project
9 expanded_conv_project_BN
10 block_1_expand
11 block_1_expand_BN
12 block_1_expand_relu
13 block_1_pad
14 block_1_depthwise
15 block_1_depthwise_BN
16 block_1_depthwise_relu
17 block_1_project
18 block_1_project_BN
19 block_2_expand
20 block_2_expand_BN
21 block_2_expand_relu
22 block_2_depthwise
23 block_2_depthwise_BN
24 block_2_depthwise_relu
25 block_2_project
26 block_2_project_BN
27 block_2_add
28 block_3_expand
29 block_3_expand_BN
30 block_3_expand_relu
31 block_3_pad
32 block_3_depthwise
33 block_3_depthwise_BN
34 block_3_depthwise_relu
35 block_3_project
36 block_3_project_BN
37 block_4_expand
38 block_4_expand_BN
39 block_4_expand_relu
40 block_4_depthwise
41 block_4_depthwise_BN
42 block_4_depthwise_relu
43 block_4_project
44 block_4_project_BN
45 block_4_add
46 block_5_expand
47 block_5_expand_BN
48 block_5_exp

In [0]:
#Freeze: this is where we will change how much to freeze

for layer in modelMNV2.layers[:]:
    layer.trainable = True

In [0]:
print(modelMNV2.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
#Optimizer

adam = keras.optimizers.Adam(lr = 1e-4)

modelMNV2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [0]:
#Early stopping

c_backs = [EarlyStopping(monitor = 'val_loss', patience = 5)]

In [0]:
#Evaluate loss before first epoch
print(modelMNV2.metrics_names)

score = modelMNV2.evaluate(X_train, Y_train, verbose = 1)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

score2 = modelMNV2.evaluate(X_valid, Y_valid, verbose = 1)
print('Validation loss:', score2[0])
print('Validation accuracy:', score2[1])

['loss', 'categorical_accuracy']
27388/27388 [==============================] - 54s 2ms/step
Train loss: 2.6923915121839816
Train accuracy: 0.14681612385094941
5262/5262 [==============================] - 10s 2ms/step
Validation loss: 2.6910232585992526
Validation accuracy: 0.1431014823431028


In [0]:
#Save model checkpoint

sys.path.append('/content/gdrive/My Drive/Colab Notebooks')
filepath = '/content/gdrive/My Drive/Colab Notebooks/modelMNV2_weights.h5'

saver = ModelCheckpoint(filepath, monitor = 'val_categorical_accuracy', verbose = 1, save_best_only = True, mode = 'max')

In [0]:
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/modelMNV2_weights_val72.h5')

Instructions for updating:
Use tf.cast instead.


In [0]:
#Train model

num_iter = 50
history = modelMNV2.fit(X_train, 
                       Y_train, 
                       epochs = num_iter, 
                       callbacks = [saver],
                       batch_size = 42, 
                       shuffle = True,  
                       verbose = 1, 
                       validation_data = (X_valid, Y_valid))
                      

Instructions for updating:
Use tf.cast instead.
Train on 27388 samples, validate on 5262 samples
Epoch 1/50
27388/27388 [==============================] - 234s 9ms/step - loss: 4.4865 - categorical_accuracy: 0.3339 - val_loss: 3.3412 - val_categorical_accuracy: 0.1650

Epoch 00001: val_categorical_accuracy improved from -inf to 0.16496, saving model to /content/gdrive/My Drive/Colab Notebooks/modelMNV2_weights.h5
Epoch 2/50
27388/27388 [==============================] - 228s 8ms/step - loss: 1.1080 - categorical_accuracy: 0.5747 - val_loss: 2.3548 - val_categorical_accuracy: 0.3786

Epoch 00002: val_categorical_accuracy improved from 0.16496 to 0.37856, saving model to /content/gdrive/My Drive/Colab Notebooks/modelMNV2_weights.h5
Epoch 3/50
27388/27388 [==============================] - 230s 8ms/step - loss: 1.0232 - categorical_accuracy: 0.6044 - val_loss: 1.7403 - val_categorical_accuracy: 0.4781

Epoch 00003: val_categorical_accuracy improved from 0.37856 to 0.47815, saving model to

In [0]:
print(history.history.keys())
#Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
print(history.history.keys())
#Summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

# Function to plot confusion matrix    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = modelMNV2.predict(X_valid)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_valid,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(7)) 

In [0]:
# Test user image
import imageio
img = imageio.imread("../content/gdrive/My Drive/sample.png")
img=np.array(img)
img=np.expand_dims(img, axis=0)
img.shape
model.predict([img])[0]

array([4.1631459e-11, 2.1360963e-06, 2.9252639e-11, 1.6000689e-11,
       1.3426913e-06, 1.3282388e-02, 9.8671418e-01], dtype=float32)

In [0]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu